**Содержание**<a id='toc0_'></a>    
- [Преобразование последовательностей (seq2seq)](#toc1_)    
  - [Основные архитектуры](#toc1_1_)    
    - [Сверточные архитектуры](#toc1_1_1_)    
    - [Трансформеры](#toc1_1_2_)    
    - [ RNN based Neural Machine Translation / RNMT+](#toc1_1_3_)    
    - [Что лучше](#toc1_1_4_)    
  - [Декодированые выходной последовательности](#toc1_2_)    
      - [Полностью жадное решение задачи поиска максимума вероятности](#toc1_2_1_1_)    
  - [Проблема низкого развнообразия генерации](#toc1_3_)    
      - [**Замена функции потерь**](#toc1_3_1_1_)    
- [Теоретические вопросы](#toc2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Преобразование последовательностей (seq2seq)](#toc0_)

Что значит "метод [seq2seq](https://en.wikipedia.org/wiki/Seq2seq)" (сокращение "sequence to sequence"). 

**На вход** подают список номеров токенов (результат токенизации исходного текста): $[t_1, ..., t_l]$ 

**На выходе** генерируем новый список номеров токенов, по которым можно будет декодировать новый результирующий текст. Таким образом, задача "sequence to sequence" — это задача генерации одного текста на основе другого текста: $[y_1, ..., y_{l_y}]$

К таким задачам, например, относится:
- машинный перевод
- диалоговые системы (генерация реплик)
- суммаизация (генерация аннотаций)
- анализ структуры (можно на seq2seq сделать POS-tagging, но не нужно!)

**Обучающая выборка** 

- Cостоит из пар (входной - выходной тексты):

    Мама мыла раму -> Mom washed the windows  
    Потом мы пошли гулять -> Then we went for a walk

- **Токенизация** (BPE, wordpieces):

$$x_{i_1,1}, x_{i_1,2}, x_{i_1,3}, ... \rightarrow y_{i_1,1},y_{i_1,2},y_{i_1,3},y_{i_1,4},.... \\ 
x_{i_2,1}, x_{i_2,2}, x_{i_2,3}, ... \rightarrow y_{i_2,1},y_{i_2,2},y_{i_2,3},y_{i_2,4},....$$

**Общий алгоритм**

В каждой seq2seq модели есть две самые главные части — это энкодер (encoder) и декодер (decoder)

1. Закодировать входную последовательность (энкодер)
- преобразует последовательность в один или несколько векторов
2. Генерировать выходную последовательность слово за словом (декодер)
- принимает в качестве входа результат работы энкодера и, на каждом шаге, выдаёт распределение вероятностей очередного токена при условии всех предыдущих
- распределение первого токена обусловлено только на входную последовательность, а все последующие — уже и на первый токен
  - $P(y_1|x_1, ...x_n)$
  - $P(y_2|x_1, ...x_n, y_1)$
  - ...
- процесс генерации продолжается до тех пор, пока декодер не сгенерирует специальный токен, обозначающий конец последовательности `<EOS>`

Такой процесс генерации называется "**авторегрессионным**".

[1] Sutskever I., Vinyals O., Le Q. V. Sequence to sequence learning with neural networks //Advances in neural information processing systems. – 2014. – С. 3104-3112.

## <a id='toc1_1_'></a>[Основные архитектуры](#toc0_)

### <a id='toc1_1_1_'></a>[Сверточные архитектуры](#toc0_)

- В фейсбуке предложили использовать архитектуру, состоящую целиком из свёрток[1,2] — свёрточные модули хорошо распараллеливаются, так как зависимости по данным — локальны и укладываются в ширину ядра свёртки. 
- Свертки инварианты к порядку в последовательности, поэтому применяется позиционное кодирование последовательностей (эмбеддинг позиций)
- Учет зависимости между токенами длины $n$ свертками с ядром $k$ требуется $O(n/k)$ сверточных слоев 
  - Если с прореживанием/dilation, то $O(\log n)$, т.е. все равно по памяти значительно хуже RNN

  **Преимущества**

  1. Такие архитектуры обладают всеми преимуществами архитектуры [Google Machine Translation](https://en.wikipedia.org/wiki/Google_Neural_Machine_Translation) (она примерно так же устроена):
    - они достаточно выразительные, мощные, и на ряде задач они дают лучшее качество, чем seq2seq модели, основанные на рекуррентках
  2. В энкодере используется двунаправленный контекст, то есть, для вычисления вектора слова на каком-то уровне, используются как соседи слева, так и соседи справа 
  3.  Свёрточные нейросети гораздо эффективнее с вычислительной точки зрения, то есть, они лучше заполняют вычислительные мощности видеокарт.

  **Недостатки**

  Дорого учитывать длинные зависимости
  - а в RNN не дорого n ячеек памяти?

  Низкое разнообразие результирующих текстов

### <a id='toc1_1_2_'></a>[Трансформеры](#toc0_)
Следующее логичное развитие — это использовать трансформер, состоящий целиком из **механизмов внутреннего внимания**.[3] 

Эта архитектура также состоит из двух частей — энкодера и декодера. 
- В энкодере используется полный механизм внимания, учитывающий все пары элементов последовательности. 
- В декодере у нас ситуация особая — когда мы генерируем очередное слово, мы не видим ничего справа. 
  - Мы можем опираться только на часть сгенерированной последовательности слева от текущего слова. 
  - Для того, чтобы в режиме обучения игнорировать часть последовательности справа от текущего слова, используется так называемый "механизм внутреннего внимания с маской". 
  - Маска применяется перед софтмаксами, внутри механизма внимания, и приводит к тому, что некоторые позиции получают нулевой вес и не участвуют в вычислении результирующих векторов. 
- Так же, как и в полносвёрточных архитектурах, здесь необходимо позиционное кодирование для того, чтобы сообщить нейросети информацию о том, где, относительно начала текста, находится каждое входное слово. 
  - На практике, часто используется синусоидальный сигнал, который складывается, добавляется к эмбеддингу токена. 
- Как мы уже говорили в лекции про трансформер и про механизмы внимания, стоимость обработки зависимостей любой длины — константна $O(1)$ и не зависит от расстояния между элементами последовательностей, между которыми зависимость существует.

Т.о. в задачах seq2seq механизм внимания:
- усиливает поток информации между энкодером и декодером (больше информации о входном тексте доходит до декодера и генерируемый текст сильнее зависит от входного текста).
- позволяет учитывать далёкие зависимости.
- позволяет избежать "забывания" начала входного текста при использовании рекуррентного энкодера.

**Преимущества**

- отлично распараллеливается
- учитывается сразу весь контекст любой длины

**Недостатки**
- сложный алгоритм обучения (?)

### <a id='toc1_1_3_'></a>[ RNN based Neural Machine Translation / RNMT+](#toc0_)

Тоже на RNN, по сути - многослойная LSTM. Тоже норм.

### <a id='toc1_1_4_'></a>[Что лучше](#toc0_)

Как обычно, где как. Например, в энторнете есть таблички где кто-то сравнивают разные варианты, и у них получилось, что лучшее качество достигается когда энкодер это трансформер, а декодер RNMT+


[1] [Gehring J. et al. Convolutional sequence to sequence learning //Proceedings of the 34th International Conference on Machine Learning-Volume 70. – JMLR. org, 2017. – С. 1243-1252.](https://arxiv.org/pdf/1705.03122.pdf)  
[2] Gehring J. et al. A convolutional encoder model for neural machine translation //arXiv preprint arXiv:1611.02344. – 2016.  
[3] [Vaswani, Ashish, et al. Attention is all you need. Advances in neural information processing systems. 2017.](https://papers.nips.cc/paper/7181-attention-is-all-you-need.pdf)

## <a id='toc1_2_'></a>[Декодированые выходной последовательности](#toc0_)

В чем проблема?

- модель предсказывается вероятности токенов
  - $P(y_1|x_1, ...x_n)$
  - $P(y_2|x_1, ...x_n, y_1)$
  - ...
- результатом работы декодера должно стать совместное распределение результирующих токенов (для выбор максимума вероятности)
$$P(y_{1}, y_{2}, ..., y_{m} | x_{1}, x_{2}, ..., x_{n}) = \prod_{j=1}^m P(y_{j} | x_{1}, x_{2}, ..., x_{n}, y_{1}, ..., y_{(j-1)})$$
- поэтому возможны варианты, как из этого распределения получать последовательности токенов (непосредственно решать **задачу генерации**):
  - семплирование из полученного распределения: $y_{1},y_{2},...,y_{m} \sim P(y_{1},y_{2},...,y_{m}|x_{1},x_{2},...,x_{n})$
  - оптимизация, поиск максимума вероятности: $y_{1},y_{2},...,y_{m} = \argmax_{y_{i}} P(y_{1}, y_{2}, ..., y_{m} | x_{1}, x_{2}, ..., x_{n})$

Т.е. имеем проблемы 
1. **баланс правдоподобия и разнообразия** (максимум вероятности / семпл из распределения)
2. задача поиска максимума по всем токенам — **[NP-полная](https://ru.wikipedia.org/wiki/NP-%D0%BF%D0%BE%D0%BB%D0%BD%D0%B0%D1%8F_%D0%B7%D0%B0%D0%B4%D0%B0%D1%87%D0%B0)**, т.е. нерешаема без полного перебора, поэтому её не представляется возможным решить за разумное время

#### <a id='toc1_2_1_1_'></a>[Полностью жадное решение задачи поиска максимума вероятности](#toc0_)

Можно применять полностью жадное решение[2] - это такое упрощение исходной задачи, когда на каждом шаге всегда выбирается наиболее правдоподобный токен, идя последовательно слева направо:

Выбираеются моды последовательности распределений:
$$ \hat y_{1} = \argmax_{y_{1}} P(y_{1}| x_{1}, x_{2}, ..., x_{m}) \\
\hat y_{2} = \argmax_{y_{2}} P(y_{2}| x_{1}, x_{2}, ..., x_{m}, \hat y_1) \\
\hat y_{3} = \argmax_{y_{3}} P(y_{3}| x_{1}, x_{2}, ..., x_{m}, \hat y_1, \hat y_2) \\
...$$

Полностью жадное решение требует $O(m - |Vocal|)$ операций.

Если модель хорошо обучилась, даже скажем прямо, переучилась, то такой алгоритм дает хорошие результаты.

#### 

Умеренно жадный алгоритм[2] категории "best first алгоритмов" поиска в графах. Направлен на поиск не оптимальных, но достаточно хороших наборов решений, и обычно дает баланс правдоподобия и разнообразия.

**Вход** - оценка условного распределния: $P(y_{i}| x_{1}, x_{2}, ..., x_{m}, y_{1}, ..., y_{i-1})$.

Нам нужна некоторая функция, которая будет оценивать качество наших последовательностей. Мы можем использовать (как раз) наш декодер для того, чтобы оценивать правдоподобие очередного токена, при условии наблюдения какой-то части последовательности.

**Выход** - последовательность токенов: $[\hat y_1, \hat y_2, ...\hat y_m]$

Это не будет конечно наилучшей оценку в смысле построенной модели, но уж точно не хуже полностью жадного алгоритма.

**Гиперпараметры алгоритма**

b - ширина луча (beamsize), порядка 10  
M - максимальная длина решения

**Алгоритм**

1. Инициализация списка частных решений (пучок лучей) и списка полных решений (в луче $b$ токенов, с которых вообще может начаться последовательность)

$$ Beam = \{ |\hat y_{i, 1}|\}, i \in \overline{1,b} \\ 
\hat y_{i,1} = \argmax_{y_{1}} P(y_{1}| x_{1}, x_{2}, ..., x_{m}) \\
Result = \emptyset $$

2. Для каждого частного решения $[\hat y_{i, 1},  .. \hat y_{i, m_i}] \in Beam$
   - для каждого токена $y_k \in Vocab$
     - построить продолжение частного решения $[\hat y_{i, 1},  .. \hat y_{i, m_i}, y_k]$
     - оценить его правдоподобие $P(\hat y_{i, 1},  .. \hat y_{i, m_i}, y_k| x_{1}, x_{2}, ..., x_{m})$
     - добавить его в луч
     - если $y_k = <EOS>$, то добавить это частное решение в $Result$

3. Удалить из луча все частные решения, кроме $b$ самых правдоподобных
4. Повтор 2-3 пока $Bean \neq \emptyset$ или в луче не останется частных решений, короче или равные по длине $M$
5. Выбрать из $Result$ наилучшее решение или несколько лучших

**Недостаток**

- предпочитает короткие решения, т.к. чем длинее последовательность, тем больше множителей (вероятностей) в оценке правдоподобия, и оценка снижается
- нужно добавлять "антиштраф" за длинные последовательности, занижать правдоподобие коротких решений

[1] [Chen, Mia Xu, et al. The best of both worlds: Combining recent advances in neural machine translation. arXiv preprint arXiv:1804.09849 (2018).](https://arxiv.org/abs/1804.09849)  
[2] Жадное декодирование и декодирование через лучевой поиск (beam-search) рассматривалось в семинаре про трансформер.

## <a id='toc1_3_'></a>[Проблема низкого развнообразия генерации](#toc0_)

Низкое разнообразие — это когда в разных ситуациях генерируется один и тот же текст. Есть множество гипотез касательно того, почему это может происходить. К основным можно отнести две:

- **слабая обусловленность на вход**, то есть связь между энкодером и декодером — слабая
  - т.е. **генератор текста работает в режиме языковой модели**
  - считается, что использование **механизма внимания** частично решает проблему
  
- **чрезмерная уверенность декодера** (английский термин: "over-confidence").[1]
  - т.е. предсказываемые вероятности токенов последовательности либо близки к 0, либо близки к 1
  - тогда попадание 0 токенов в луч обнуляет правдоподобие и последовательность отбрасывается
  - возможный способ борьбы с чрезмерной уверенностью — это **замена функции потерь**

#### <a id='toc1_3_1_1_'></a>[**Замена функции потерь**](#toc0_)

Исходная функция потерь для токена $y_i$ - категориальная кросс-энтропия:
$$CE(y_i, y_y^{GT}) = -\log NNet (x_1, ..., x_m, y_1, ...,y_{i-1})_{y_i^{GT}} = \\
= - \sum_{k=1}^{|Vocab|} [k = y_i^{GT}] \log NNet(x_1, ..., x_m, y_1, ...,y_{i-1})$$

$[k = y_i^{GT}]$ - индикаторная функция, равна 1, если k совпадает с номером истинного токена (таргета), иначе 0

1. **Сглаживание меток классов** (label smoothing[2])

$$SmoothedCE(y_i, y_y^{GT}) = - \sum_{k=1}^{|Vocab|} W(k, y_i^{GT}) \log NNet(x_1, ..., x_m, y_1, ...,y_{i-1}) \\ 
W(k, y_i^{GT}) = \begin{cases} 1 - \beta & k = y^{GT} \\ \beta \cdot P(y^{GT}) & k \neq y^{GT} \end{cases}$$

$W(k, y_i^{GT})$ - мягкая индикаторная функция, $\beta \sim 0.1$. Т.е. для правильного токена вес будет браться "почти" 1, а для остальных - пропорционально априорной вероятности правильного токена в тексте.

2. **Энтропийная регуляризация** - штраф за контрастность

$$ loss(y_i, y_y^{GT}) = CE(y_i, y_y^{GT}) - \beta H (NNet (x_1, ..., x_m, y_1, ...,y_{i-1})_{y_i^{GT}}) \\
H(P) = -\sum_{i=1}^K p_i \log p_i$$

- энтропия максимальна, когда распределение равномерное, а минимальна — когда распределение вырождено


Семинар в директории `./stepik-dl-nlp`

# <a id='toc2_'></a>[Теоретические вопросы](#toc0_)

1. Алгоритм машинного перевода переводит текст с английского языка на русский. Размер английского словаря равен 5000 токенов, русского - 8000. Оцените, сколько последовательностей рассмотрит декодер при генерации перевода наибольшей длины 5 при условии, что при декодировании используется алгоритм полного перебора?

- Число размещений **с повторениями** из 8000 по 5: $A_n^k = n^k = 8000^5 = 32768000000000000000$ 

2. Алгоритм машинного перевода работает с переводом текста с английского языка на русский. Размер английского словаря равен 5000 токенов, русского - 8000. Оцените, сколько последовательностей сгенерирует декодер при переводе предложения длины 5 при условии, что используется лучевой поиск (beam search) при b=3 (b - ширина луча)? 

- Дерево высоты 5 и по 3 ветви в узлах, сколько элементов на последнем уровне? $3^0, 3^1, ... , 3^4, 3^5$, причем нулевой уровень, это еше не дерево по данному алгоритму

3. Пусть у нас есть предложение, состоящее из произвольных цифр. Например, '1576429830'. Посчитайте **перплексию** этого предложения относительно модели, которая считает, что вероятность встретить каждую цифру равна 0.1.
Формула перплексии:

$$PP(W) = P(w_1, w_2, .., w_N)^{-\frac{1}{N}} = \sqrt[N]{\dfrac{1}{P(w_1, w_2, .., w_N)}} = \sqrt[N]{\dfrac{1}{\prod_{i=1}^N P(w_i | w_1, .., w_{i-1})}} = \sqrt[10] {1 / 0.1^{10}}= 10^1$$

4. **BLEU (bilingual evaluation understudy)** - метрика для оценивания качества машинного перевода, основанная на сравнении перевода, предложенного алгоритмом, и референсного перевода (ground truth). Сравнение производится на основе подсчета n-грамм (n меняется от 1 до некоторого порога, например, 4), которые встретились и в предложенном переводе, и в референсном (ground truth). После подсчета совстречаемости n-грамм полученная метрика умножается на так называемый **brevity penalty** - штраф за слишком короткие варианты перевода. Brevity penalty считается как <количество слов в переводе, предложенном алгоритмом> / <количество слов в референсном переводе>.

Формула:

$$BLEU = \text{brevity penalty} \cdot \left (\prod_{i=1}^n \text{precision}_i \right)^{1/n} \cdot 100\% = (6/8*4/7*2/6)**(1/3) * 100$$
где $\text{brevity penalty} = min \left(1, \dfrac{\text{output length}}{\text{reference length}} \right)$

- Перевод, предложенный алгоритмом: "Кошка вышла из дома и села на крыльцо"
- Референсный перевод (ground truth): "Кошка вышла из комнаты и села на ступеньки"
- $BLEU =  = 1 * (6/8*4/7*2/6)^{1/3} * 100 \% = 52 \%$
